In [5]:
import dill

In [1]:
%run mp.py

started
finished
267.73433780670166
10


In [6]:
simulation_data = results.copy()
with open("results_object.pkl", "wb") as dill_file:
    dill.dump(simulation_data, dill_file)

In [7]:
with open("results_object.pkl", "rb") as dill_file:
    simulation_data2 = dill.load(dill_file)

In [12]:
simulation_data2[(5,0)].graph.nodes[(5,0)]['color']

1

In [5]:
connected_components = nx.connected_components(subgraph)

In [2]:
initialize()

In [102]:
len(list(connected_components))

0